In [1]:
import math
from pathlib import Path

import arcpy
from ba_tools import data
import pandas as pd
import numpy as np

from h3 import h3
from itertools import chain
import json
import os
from pathlib import Path
import sys
from copy import deepcopy

from arcgis.features import GeoAccessor
from arcgis.geometry import Geometry
import pandas as pd

project_parent = Path('./').absolute().parent

data_dir = project_parent/'data'

data_raw = data_dir/'raw'
data_int = data_dir/'interim'
data_out = data_dir/'processed'

gdb_raw = data_raw/'raw.gdb'
gdb_int = data_int/'interim.gdb'
gdb_out = data_out/'processed.gdb'

# import the project package from the project package path
sys.path.append(str(project_parent/'src'))
import h3_arcgis

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

In [57]:
# pth_bp = Path(r'D:\arcgis\ba_data\Data\Demographic Data\block_data.gdb\USA_ESRI_2019_blocks')
pth_bp = gdb_int/'sea_block_points'
pth_bp

df_bp = GeoAccessor.from_featureclass(str(pth_bp))
df_bp.head()

df_bp = df_bp[['FIPS', 'SHAPE', 'HH_C', 'POP_C']].copy()
df_bp.columns = ['FIPS', 'SHAPE', 'HouseholdCount', 'Population']

pt_df = df_bp.copy()
pt_df.head()

,FIPS,SHAPE,HouseholdCount,Population
0,530530729061083,"{'x': -122.70289379994466, 'y': 47.07056900018...",0.0,0.0
1,530530730061116,"{'x': -122.38923279983811, 'y': 46.84712710009...",2.0,6.0
2,530530732003038,"{'x': -122.27535699972861, 'y': 46.83756129998...",8.0,15.0
3,530530732003040,"{'x': -122.31008460020831, 'y': 46.84448979965...",2.0,4.0
4,530530732003041,"{'x': -122.25264919960182, 'y': 46.84812859970...",1.0,1.0


In [58]:
hex_df = GeoAccessor.from_featureclass(str(gdb_int/f'sea_hex'))
hex_df.head()

,OBJECTID,h3_id,Population,h3_lvl,SHAPE
0,1,8528d557fffffff,1862,5.0,"{'rings': [[[-122.36929827899996, 47.500677578..."
1,2,8528d0b7fffffff,1555,5.0,"{'rings': [[[-122.20382834499998, 47.866673952..."
2,3,8628d5427ffffff,946,6.0,"{'rings': [[[-122.26493069499998, 47.570222506..."
3,4,8528d437fffffff,809,5.0,"{'rings': [[[-121.96960155399995, 47.252683501..."
4,5,8528d4affffffff,710,5.0,"{'rings': [[[-121.96013460199998, 47.111783662..."


In [67]:
pt_bg = df_bp.copy()

h3_lvl_max = int(hex_df.h3_lvl.max())
h3_lvl_min = int(hex_df.h3_lvl.min())

h3_id_df = h3_arcgis.add_h3_ids_to_points(hex_df, hex_lvl_max, hex_lvl_min)

hex_h3_id_lst = list(hex_df['h3_id'].unique())

h3_range = list(range(hex_lvl_min, hex_lvl_max+1))
h3_range.sort(reverse=True)

for h3_col in [h3_arcgis._h3_col(lvl) for lvl in h3_range]:
    
    h3_sel = h3_id_df[h3_col].isin(hex_h3_id_lst)
    
    h3_id_df.loc[h3_sel, 'h3_id'] = h3_id_df[h3_sel][h3_col]
    break

In [68]:
h3_id_df[h3_id_df['Population'] < 1000]

,OBJECTID,h3_id,Population,h3_lvl,SHAPE,h3_08,h3_07,h3_06,h3_05
2,3,8628d5427ffffff,946,6.0,"{'rings': [[[-122.26493069499998, 47.570222506...",8828d54201fffff,8728d5420ffffff,8628d5427ffffff,8528d543fffffff
3,4,8528d437fffffff,809,5.0,"{'rings': [[[-121.96960155399995, 47.252683501...",8828d43401fffff,8728d4340ffffff,8628d4347ffffff,8528d437fffffff
4,5,8528d4affffffff,710,5.0,"{'rings': [[[-121.96013460199998, 47.111783662...",8828d4ac01fffff,8728d4ac0ffffff,8628d4ac7ffffff,8528d4affffffff
5,6,8528d5cffffffff,603,5.0,"{'rings': [[[-122.35916231899995, 47.360476374...",8828d5cc01fffff,8728d5cc0ffffff,8628d5cc7ffffff,8528d5cffffffff
6,7,8628d5457ffffff,590,6.0,"{'rings': [[[-122.35589532399996, 47.692889724...",8828d54501fffff,8728d5450ffffff,8628d5457ffffff,8528d547fffffff
...,...,...,...,...,...,...,...,...,...
451,452,8828d55299fffff,9,8.0,"{'rings': [[[-122.35295626799996, 47.490610912...",8828d55299fffff,8728d5529ffffff,8628d552fffffff,8528d553fffffff
452,453,8828d54e85fffff,9,8.0,"{'rings': [[[-122.33036157599997, 47.772551403...",8828d54e85fffff,8728d54e8ffffff,8628d54efffffff,8528d54ffffffff
453,454,8628d5d4fffffff,8,6.0,"{'rings': [[[-122.64716500199995, 47.203495581...",8828d5d481fffff,8728d5d48ffffff,8628d5d4fffffff,8528d5d7fffffff
454,455,8828d5d0adfffff,8,8.0,"{'rings': [[[-122.42803695099997, 47.182686053...",8828d5d0adfffff,8728d5d0affffff,8628d5d0fffffff,8528d5d3fffffff


In [69]:
count_srs = h3_id_df['h3_id'].value_counts()
count_srs[count_srs > 1]

Series([], Name: h3_id, dtype: int64)